<a href="https://colab.research.google.com/github/HuyCE/PPO_fliping_coin_with_random_randint/blob/main/Dump_coin_game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q swig
!pip install -q gymnasium[box2d]
!pip install stable-baselines3[extra]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import gymnasium as gym
from gymnasium.spaces import Box, Discrete, Dict
import numpy as np
from typing import Optional
import random

class Casinoike(gym.Env):

    def __init__(self, size: int = 5):
        self.wallet = 100 # in [USD]
        self.bet = 0
        self.host_wallet = 10000
        self.host_win = 0
        self.bot_win = 0
        self.number_of_rounds = 0
        # self.reward = 0
        # Observations are dictionaries with the agent's and the target's location.
        # Each location is encoded as an element of {0, ..., `size`-1}^2
        self.observation_space = gym.spaces.Dict(
            {
                "wallet": Box(0, 10100, shape=(1,), dtype=np.int64),
            }
        )
        # Action is bet money at each play
        self.action_space = Box(0,1, shape=(2,))
        # action [0]: bet - [1] bet money
    def _get_obs(self):
        return {"wallet": np.array([self.wallet], dtype=np.int64)}

    def _get_info(self):
        return {
            "bet_money": self.bet,
            "host_wallet": self.host_wallet
        }

    def reset(self, seed: Optional[int] = None, options: Optional[dict] = None):
        # We need the following line to seed self.np_random
        super().reset(seed=seed)

        self.wallet = 100 # in [USD]
        self.bet = 0
        self.host_wallet = 10000
        # self.reward = 0

        observation = self._get_obs()
        info = self._get_info()

        return observation, info

    def step(self, action):
        self.bet = int(action[-1]*100)+1
        #all-in condition
        if(self.bet >= self.wallet):
          self.bet = self.wallet
          if(self.bet > 101):
            print("error here when self bet is: ", self.bet)

        # self.wallet = self.wallet - self.bet

        # host flip the coin
        flip_result = random.randint(0,1)
        # check result
        if (round(action[0]) == flip_result):
          self.wallet += self.bet
          self.host_wallet -= self.bet
          # print("wallet up: ", self.wallet)
        else:
          self.wallet -= self.bet
          self.host_wallet += self.bet
        # Dev log
        # print(f"Bet: {self.bet} on {round(action[0])}")
        reward = self.wallet
        terminated = False
        # An environment is completed if and only if the agent has reached the target
        # terminated = np.array_equal(self._agent_location, self._target_location)
        if self.wallet <= 0 or self.host_wallet <= 0:
          if self.wallet <= 0:
            # print("You broke")
            self.host_win += 1
          elif self.host_wallet <= 0:
            # print("Host broke")
            self.bot_win +=1
          else:
            print("error")
          self.number_of_rounds += 1

          terminated = True

        truncated = False
        observation = self._get_obs()
        info = self._get_info()

        return observation, reward, terminated, truncated, info

In [ ]:
from stable_baselines3.common.env_checker import check_env
env = Casinoike()


In [ ]:
check_env(env)

You broke


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/env_checker.py:461: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf. https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  warnings.warn(


In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback


class CustomCallback(BaseCallback):
    def __init__(self, verbose: int = 0):
        super().__init__(verbose)
        self.best_global_score = 0
        self.bot_win = 0
        self.host_win = 0
        self.games = 0
    def _on_training_start(self) -> None:
        """
        This method is called before the first rollout starts.
        """
        pass

    def _on_rollout_start(self) -> None:
        """
        A rollout is the collection of environment interaction
        using the current policy.
        This event is triggered before collecting new samples.
        """
        self.best_local_res = 0
        self.wallet_value = 0

    def _on_step(self) -> bool:
        """
        This method will be called by the model after each call to `env.step()`.

        For child callback (of an `EventCallback`), this will be called
        when the event is triggered.

        :return: If the callback returns False, training is aborted early.
        """
        self.wallet_value = self.training_env.get_attr("wallet")[0]
        if(self.wallet_value > self.best_local_res):
          self.best_local_res = self.wallet_value
        return True

    def _on_rollout_end(self) -> None:
        """
        This event is triggered before updating the policy.
        """
        if(self.best_global_score < self.best_local_res):
          self.best_global_score = self.best_local_res
          print("New best wallet value: ", self.best_global_score)

    def _on_training_end(self) -> None:
        """
        This event is triggered before exiting the `learn()` method.
        """
        self.host_win = self.training_env.get_attr("host_win")
        self.bot_win = self.training_env.get_attr("bot_win")
        self.games = self.training_env.get_attr("number_of_rounds")
        print(f"Sum up win rate (host/bot/game): {self.host_win}/{self.bot_win}/{self.games}")


env = Casinoike()
model = PPO("MultiInputPolicy", env)
custom_cb = CustomCallback()
model.learn(total_timesteps=100000, callback=custom_cb)


New best wallet value:  4486
New best wallet value:  5061
New best wallet value:  8562
New best wallet value:  9094
New best wallet value:  10050
New best wallet value:  10098
Sum up win rate (host/bot/game): [190]/[5]/[195]
